<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Continue_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install keras_bert

In [3]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-jbjyz71v
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-jbjyz71v
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=eef2a5ae466455790eba03e318295509d333411b611d9e87f90b92d17ab07c61
  Stored in directory: /tmp/pip-ephem-wheel-cache-d_k5wye2/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [5]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

fatal: destination path 'bachelor_design' already exists and is not an empty directory.
/content/bachelor_design


In [6]:
!git checkout equal_realize
!git pull

Already on 'equal_realize'
Your branch is up to date with 'origin/equal_realize'.
Already up to date.


# Mount GDriver
Perpare Colab environment.

In [7]:
from driver_amount import addh

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
[Colab] Using address head: /gdrive/My Drive


# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [9]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [12]:
tag_vocab

{'': 0,
 'B-EI': 1,
 'B-EIF': 10,
 'B-EO': 4,
 'B-EQ': 8,
 'B-ILF': 6,
 'I-EI': 2,
 'I-EIF': 11,
 'I-EO': 5,
 'I-EQ': 9,
 'I-ILF': 7,
 'O': 3}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [14]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [17]:
model.load_weights(addh + config.MODEL_PATH)

# Load train record

In [0]:
import json
with open(addh + config.TRAIN_REC_PATH, "r") as fd:
    train_rec = json.load(fd)
last_epoch = 0
if len(train_rec) > 1:
    last_epoch = train_rec[-1]["epoch"]

# Fine-tuning

In [19]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
model.fit(
    x=[train_token_id_seqs, train_segment_seqs],
    y=[train_tag_id_seqs], 
    batch_size=2,
    initial_epoch=last_epoch+1,
    epochs=train_rec[0]["train_params"]["epochs"],
    verbose=1,
    validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
    callbacks=[epoch_callback]
)

Train on 7005 samples, validate on 779 samples
Epoch 5/15
7005/7005 [==============================] - 4899s 699ms/step - loss: 56.1067 - crf_viterbi_accuracy: 0.8872 - val_loss: 52.8646 - val_crf_viterbi_accuracy: 0.8885
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 191s 245ms/step
{'EO': 0.9852579852579852, 'ILF': 0.6388888888888888, 'EI': 0.794392523364486, 'EQ': 0.76, 'EIF': 0.8888888888888888}
-cost_precision: 1.013486  -label_precision: 0.830552
Epoch 6/15
7005/7005 [==============================] - 4923s 703ms/step - loss: 56.0349 - crf_viterbi_accuracy: 0.9043 - val_loss: 52.8567 - val_crf_viterbi_accuracy: 0.8971
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 196s 252ms/step
{'EO': 0.9287469287469288, 'ILF': 0.4722222222222222, 'EI': 0.7850467289719626, 'EQ': 0.4, 'EIF': 0.8888888888888888}
-cost_precision: 0.894981  -label_precision: 0.789474
Epoch 7/15
 840/7005 [==>..

In [0]:
model.save(addh + config.MODEL_PATH)